In [1]:
%matplotlib inline 
%load_ext autoreload 
%autoreload 2

from __future__ import (print_function, division, absolute_import)

import os
import glob
import copy
import sys
import warnings

import numpy as np

# Matplotlib related
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)

from astropy.table import \
    Table, \
    Column, \
    vstack, \
    unique
    
from astropy.utils.metadata import MergeConflictWarning
warnings.filterwarnings('ignore', category=MergeConflictWarning, 
                        append=True)
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.utils.console import ProgressBar

from numpy import nditer

In [2]:
#import tables
decals_table_g09 = Table.read('decals_dr3_g09_short.fits', format='fits')
hsc_table_g09 = Table.read('hsc_s16a_fastlane_g09_short (3).fits', format='fits')

In [3]:
#sort tables in order of RA for ease of search
decals_table_g09.sort('RA')
hsc_table_g09.sort('ra')

In [4]:
#get RA data from each table
hsc_RA = hsc_table_g09['ra']
decal_RA = decals_table_g09['RA']


In [5]:
# number of hsc galaxies << number of decals, so look for hsc in decals
#for each element in hsc_RA, find array of values within certain amount of degrees in decal

deg = .01
i = 0
j = len(decal_RA)-1

lower_array_of_ranges_RA = []
upper_array_of_ranges_RA = []
for x in np.nditer(hsc_RA):
    #find lower limit
    while decal_RA[i] < x-deg:
        i=i+1
    upper_array_of_ranges_RA.append(i)
    #find upper limit
    while decal_RA[j] > x+deg:
        j = j-1
    lower_array_of_ranges_RA.append(j)
    
print ("lower array of ranges: " + str(lower_array_of_ranges_RA[50]))
print ("upper array of ranges: " + str(upper_array_of_ranges_RA[50]))
print ("lower range: " + str(decal_RA[lower_array_of_ranges_RA[50]]))
print ("upper range: " + str(decal_RA[upper_array_of_ranges_RA[50]]))

lower array of ranges: 9270
upper array of ranges: 44631
lower range: 128.491602066
upper range: 129.353590636


In [6]:
#get DEC data from each table
hsc_dec = hsc_table_g09['dec']
decal_dec = decals_table_g09['DEC']

In [ ]:
final_group = []
indcs = np.array([])
decalID_in_order = [0]*len(hsc_dec)

#find the galaxy with the minimum dec difference
for i in range(len(hsc_dec)):
    minDiff = hsc_dec[i]
    for j in range(lower_array_of_ranges_RA[i], upper_array_of_ranges_RA[i]):
        if minDiff > abs (hsc_dec[i] - decal_dec[j]):
            minDiff = abs(hsc_dec[i] - decal_dec[j])
            minVal = decal_dec[j]
            min_idx = j
    #get id in decal catalog
    decal_id = decals_table_g09['OBJID'][j]
    #put this in the hsc catalog in new column
    decalID_in_order[i] = decal_id
    print (str(i) + ", " + str(decal_id))
    
col = Column(decalID_in_order, name = 'decalId')
hsc_table_g09.add_column(col)

0, 3066
1, 3066
2, 1869
3, 3750
4, 4260
5, 2026
6, 845
7, 546
8, 2044
9, 3879
10, 1475
11, 1763
12, 2175
13, 1772
14, 669
15, 114
16, 2544
17, 379
18, 1152
19, 3058
20, 787
21, 1140
22, 3156
23, 2449
24, 452
25, 3142
26, 262
27, 3107
28, 1776
29, 2504
30, 1953
31, 2436
32, 2624
33, 2086
34, 1156
35, 640
36, 1674
37, 1467
38, 1945
39, 2720
40, 2876
41, 1468
42, 1330
43, 577
44, 2522
45, 2344
46, 2456
47, 1227
48, 1091
49, 2013
50, 437
51, 385
52, 416
53, 906
54, 1088
55, 3196
56, 2645
57, 1897
58, 2167
59, 1300
60, 1946
61, 1528
62, 1977
63, 817
64, 3157
65, 1901
66, 1010
67, 461
68, 2994
69, 1056
70, 792
71, 2494
72, 688
73, 141
74, 1520
75, 2272
76, 1733
77, 2070
78, 1688
79, 1185
80, 1122
81, 333
82, 328
83, 2705
84, 1334
85, 1175
86, 3734
87, 1007
88, 1012
89, 3129
90, 798
91, 376
92, 1275
93, 693
94, 2391
95, 620
96, 1301
97, 2827
98, 1904
99, 211
100, 256
101, 3396
102, 2487
103, 696
104, 3395
105, 3091
106, 3269
107, 2633
108, 2633
109, 672
110, 153
111, 144
112, 1483
113, 1089
1